In [8]:
import pandas as pd

In [9]:
df = pd.read_csv('BTC_ETH_LTC_1week_1m.csv')

In [10]:
def sma(data, window):
    return(data.rolling(window = window).mean())

def bollinger_band(data, sma, window, nstd):
    std = data.rolling(window = window).std()
    upper_band = sma + std*nstd
    lower_band = sma - std*nstd
    return upper_band, lower_band


symbols = ['BTC', 'ETH', 'LTC']


nstd = 3

for symbol in symbols:
    df[f'{symbol}_SMA'] = sma(df[f'{symbol}-USD_Open'], 20)
    df[f'{symbol}_Upper_Band'], df[f'{symbol}_Lower_Band'] = bollinger_band(df[f'{symbol}-USD_Open'], df[f'{symbol}_SMA'], 20, nstd)


df.dropna(inplace=True)

In [11]:
class TradingEnv:
    def __init__(self, balance_amount, balance_unit, symbols):
        self.balance_amount = balance_amount
        self.balance_unit = balance_unit
        self.buys = []
        self.sells = []
        self.trading_fee_multiplier = 0.99925      # VIP level 0, paying fees with BNB
        self.symbols = symbols


        self.bottoms = {}
        self.reset_bottoms()

        self.tops = {}
        self.reset_tops()

    
    def buy(self, symbol, buy_price, time):
        self.balance_amount = (self.balance_amount / buy_price) * self.trading_fee_multiplier
        self.balance_unit = symbol
        self.buys.append([symbol, time, buy_price])

    def sell(self, sell_price, time):
        self.balance_amount = (self.balance_amount * sell_price) * self.trading_fee_multiplier
        self.sells.append([symbol, time, sell_price])
        self.balance_unit = 'BUSD'

    def reset_bottoms(self):
        for symbol in self.symbols:
            self.bottoms[symbol] = 'None'

    def reset_tops(self):
        for symbol in self.symbols:
            self.tops[symbol] = 'None'


In [12]:
'''
Initially, when price is within channel = none
As soon as we cross first bottom = hit
As soon as we cross above bottom after hit = released
'''

'\nInitially, when price is within channel = none\nAs soon as we cross first bottom = hit\nAs soon as we cross above bottom after hit = released\n'

In [13]:
env = TradingEnv(balance_amount=100, balance_unit='BUSD', symbols=symbols)
for i in range(len(df)):    
    if env.balance_unit == 'BUSD':  # Want to buy

        for symbol in symbols:
            print(env.balance_amount, env.balance_unit)
            if env.bottoms[symbol] == 'hit' and (df[f'{symbol}-USD_Low'].iloc[i] > df[f'{symbol}_Lower_Band'].iloc[i]):
                env.bottoms[symbol] = 'released'

            if df[f'{symbol}-USD_Low'].iloc[i] <= df[f'{symbol}_Lower_Band'].iloc[i]:  # Buy Signal
                if env.bottoms[symbol] == 'released':
                    env.buy(symbol, buy_price=df[f'{symbol}_Lower_Band'].iloc[i], time=df['OpenTime'].iloc[i])
                    env.reset_bottoms()
                    break
                else: 
                    env.bottoms[symbol] = 'hit' 



    if env.balance_unit != 'BUSD':  # Want to sell
        if env.tops[env.balance_unit] == 'hit' and (df[f'{env.balance_unit}-USD_High'].iloc[i] < df[f'{symbol}_Upper_Band'].iloc[i]):
            env.tops[env.balance_unit] = 'released'
        if df[f'{env.balance_unit}-USD_High'].iloc[i] >= df[f'{symbol}_Upper_Band'].iloc[i]:
            if env.tops[env.balance_unit] == 'released':
                env.sell(sell_price=df[f'{symbol}_Upper_Band'].iloc[i], time=df['OpenTime'].iloc[i])
                env.reset_tops()
            else:
                env.tops[env.balance_unit] = 'hit'


if env.balance_unit != 'BUSD':
    env.sell(df[f'{env.balance_unit}-USD_Close'].iloc[-1], df['CloseTime'].iloc[-1])
print(env.balance_amount, env.balance_unit)

100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
100 BUSD
1

In [14]:
env.tops

{'BTC': 'None', 'ETH': 'None', 'LTC': 'None'}